<a href="https://colab.research.google.com/github/YorkJong/vistock/blob/feature%2Fibd/notebooks/ibd_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with IBD RS Rating, inspired by the Investor's Business Daily (IBD) methodology.

### Install vistock from GitHub (免費版Colab會固定時間清掉安裝的東西，所以重安裝是新連線後最先要做的事)

In [3]:
%pip install "git+https://github.com/YorkJong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/YorkJong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-1n_ejeqg
  Running command git clone --filter=blob:none --quiet https://github.com/YorkJong/vistock.git /tmp/pip-req-build-1n_ejeqg
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/YorkJong/vistock.git to commit e00628b830361c0021df97305d3e06f5f7b56345
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 1.1 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.4.0-py3-none-any.whl size=42026 sha256=e35919b882b565f434ca5bfd3332d3e1d4e9963e790102a0234247d4d909facd
  Stored in directory: /tmp/pip-ephem-wheel-cache-m7ykeati/wheels/3e/1c/74/97ec620813c1bb106679625869f7409ca5be8735d0a74457bf
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4

### Setup and Configuration

In [4]:
# install cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [5]:
import os
from datetime import datetime

from vistock import ibd
from vistock import tw
from vistock.stock_indices import get_tickers


def rank(code, min_percentile=90, period='2y',
         tickers_getter=get_tickers, ref_ticker='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)
    rank_stock, rank_indust = ibd.rankings(tickers, period=period,
                                           ref_ticker=ref_ticker)

    if rank_stock.empty or rank_indust.empty:
        print("Not enough data to generate rankings.")
        return

    print('\nStock Rankings:')
    #display(rank_stock)
    top_stocks = rank_stock[rank_stock[ibd.TITLE_PERCENTILE] >= min_percentile]
    display(top_stocks)
    num_rows, _ = top_stocks.shape
    print(f'\nnumber of filtered tickers: {num_rows}')
    top_stock_list = list(top_stocks["Ticker"])
    if tickers_getter is tw.get_tickers:
        top_stock_list = [tw.stock_name(ticker) for ticker in top_stock_list]
    print(top_stock_list)

    print('\n\nIndustry Rankings:')
    top_industries = rank_indust[rank_indust[ibd.TITLE_PERCENTILE] >= min_percentile]
    display(top_industries)
    num_rows, _ = top_industries.shape
    print(f'\nnumber of filtered sectors: {num_rows}')

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Save to CSV
    print("\n\n***")
    today = datetime.now().strftime('%Y%m%d')
    for table, kind in zip([rank_stock, rank_indust],
                           ['stocks', 'industries']):
        filename = f'{code}_{kind}_{period}_{today}.csv'
        table.to_csv(os.path.join(out_dir, filename), index=False)
        print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")


### Usage Explanation

##### Parameters
source (選擇要分析的股票來源):
- The source of stocks to analyze

min_percentile (最小百分位)
- The minimum percentile for a stock to be included in the rankings.

period (歷史資料時間範圍)：
- The period for which to fetch historical data.

### IBD RS Rating and Ranking

In [6]:
source = "All Indices" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "PHLX Semiconductor", "All Indices"]
min_percentile = 90 # @param {"type":"slider","min":1,"max":99,"step":1}
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'PHLX Semiconductor': 'SOX',
    'All Indices': 'SPX+DJIA+NDX+SOX',
}

rank(code_from_name[source], min_percentile, period)

ERROR:yfinance:$BF.B: possibly delisted; No price data found  (period=2y)


$BF.B: possibly delisted; No price data found  (period=2y)


ERROR:yfinance:ARM: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:SW: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:SOLV: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:GEV: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:VLTO: Period '2y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', 'ytd', 'max']
ERROR:yfinance:BRK.B: No data found, symbol may be delisted



Stock Rankings:


,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
315,VST,Utilities,Utilities - Independent Power Producers,156.67,192.00,238.58,140.22,99,99,99,97,1
17,NVDA,Technology,Semiconductors,148.98,187.10,169.40,177.27,99,99,99,99,2
50,IRM,Real Estate,REIT - Specialty,144.50,124.08,116.53,103.70,99,95,88,69,3
62,FICO,Technology,Software - Application,141.96,131.59,119.32,131.11,99,97,91,96,4
389,GDDY,Technology,Software - Infrastructure,140.46,127.27,132.61,118.52,99,97,97,92,5
411,HWM,Industrials,Aerospace & Defense,135.26,121.73,134.66,114.15,99,94,98,88,6
103,MMM,Industrials,Conglomerates,134.78,104.54,112.07,81.53,98,80,83,13,7
229,NRG,Utilities,Utilities - Independent Power Producers,134.10,135.64,171.52,116.56,98,98,99,90,8
198,AXON,Industrials,Aerospace & Defense,132.67,99.26,113.24,113.21,98,68,84,86,9
484,TRGP,Energy,Oil & Gas Midstream,131.73,125.76,122.31,106.44,98,96,93,75,10



number of filtered tickers: 52
['VST', 'NVDA', 'IRM', 'FICO', 'GDDY', 'HWM', 'MMM', 'NRG', 'AXON', 'TRGP', 'KKR', 'GE', 'UHS', 'TSM', 'MHK', 'ANET', 'ISRG', 'MPWR', 'NTAP', 'TYL', 'RCL', 'K', 'VTR', 'AVGO', 'EFX', 'CBRE', 'BRO', 'COST', 'CFG', 'WELL', 'META', 'HCA', 'CEG', 'GS', 'EXR', 'MSI', 'GRMN', 'IP', 'CTAS', 'DHI', 'FITB', 'TMUS', 'LLY', 'TT', 'PGR', 'HIG', 'GEN', 'PM', 'RTX', 'ICE', 'EBAY', 'FIS']


Industry Rankings:


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
74,Utilities - Independent Power Producers,Utilities,145.38,163.82,205.05,128.39,"VST,NRG",99,99,99,98,1
5,Medical Care Facilities,Healthcare,120.42,101.34,109.72,107.30,"UHS,HCA,DVA",98,89,92,83,2
19,Oil & Gas Midstream,Energy,116.81,108.45,111.29,96.99,"TRGP,KMI,WMB,OKE",97,96,95,49,3
49,Conglomerates,Industrials,115.20,99.05,102.60,85.44,"MMM,HON",96,75,67,12,4
64,REIT - Healthcare Facilities,Real Estate,115.14,99.87,99.82,87.93,"VTR,WELL,DOC",95,82,61,21,5
71,Tobacco,Consumer Defensive,114.32,100.10,98.45,84.59,"PM,MO",94,84,55,8,6
46,Residential Construction,Consumer Cyclical,112.72,100.24,107.74,111.51,"DHI,PHM,NVR,LEN",93,85,87,94,7
79,Consulting Services,Industrials,109.78,99.39,96.46,104.24,"EFX,VRSK",92,78,43,73,8
11,Aerospace & Defense,Industrials,108.91,97.61,106.78,102.02,"HWM,AXON,GE,RTX,LMT,TDG,LHX,GD,NOC,HII,TXT,BA",91,67,83,64,9



number of filtered sectors: 9


***
Your "SPX+DJIA+NDX+SOX_stocks_2y_20240808.csv" is in the "out" folder.
Your "SPX+DJIA+NDX+SOX_industries_2y_20240808.csv" is in the "out" folder.
***



### IBD RS Rating and Ranking for Taiwan Stocks

In [7]:
from vistock import tw

source = "上市+上櫃" #@param ["上市", "上櫃", "上市+上櫃", "興櫃", "全部"]
min_percentile = 95 # @param {"type":"slider","min":1,"max":99,"step":1}
period = "1y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '上市+上櫃': 'TWSE+TPEX',
    '興櫃': 'ESB',
    '全部': 'TWSE+TPEX+ESB'
}

rank(code_from_name[source], min_percentile, period,
     tickers_getter=tw.get_tickers, ref_ticker='^TWII')

ERROR:yfinance:6928.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:00949.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', 'ytd', 'max']
ERROR:yfinance:020038.TW: Period '1y' is invalid, must be one of ['1d', '5d']
ERROR:yfinance:020018.TW: Period '1y' is invalid, must be one of ['1d', '5d']
ERROR:yfinance:020020.TW: Period '1y' is invalid, must be one of ['1d', '5d']
ERROR:yfinance:6949.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:6423.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:020037.TW: Period '1y' is invalid, must be one of ['1d', '5d']
ERROR:yfinance:6914.TW: Period '1y' is invalid, must be one of ['1d', '5d', '1mo', '3mo', '6mo', 'ytd', 'max']
ERROR:yfinance:020012.TW: Period '1y' is invalid, must be one of ['1d', '5d']
ERROR:yfinance:6771.TW: Period '1y' is invalid, must be one o


Stock Rankings:


,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
209,6144.TWO,Communication Services,Entertainment,473.76,231.21,117.40,92.81,99.0,99,93,16,1
1174,1799.TWO,Healthcare,Drug Manufacturers—Specialty & Generic,415.14,269.68,137.98,118.08,99.0,99,98,95,2
1362,8374.TW,Industrials,Industrial Distribution,361.59,245.99,93.33,92.35,99.0,99,53,13,3
1662,2365.TW,Technology,Computer Hardware,356.58,238.12,109.85,124.25,99.0,99,88,97,4
630,6442.TW,Technology,Electronic Components,356.02,315.55,141.33,106.66,99.0,99,98,87,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1982,6212.TWO,Real Estate,Real Estate - Development,147.95,108.15,112.26,97.46,95.0,84,90,53,97
1458,9946.TW,Real Estate,Real Estate - Diversified,147.92,140.90,130.91,112.44,95.0,96,97,93,98
378,9105.TW,Technology,Computer Hardware,147.17,152.87,92.69,108.34,95.0,97,50,89,99
1746,4127.TWO,Healthcare,Drug Manufacturers - Specialty & Generic,146.63,141.80,181.54,97.01,95.0,96,99,49,100



number of filtered tickers: 101
['得利影', '易威', '羅昇', '昆盈', '光聖', '穎漢', '均華', '皇昌', '新復興', '海悅', '晶彩科', '欣巴巴', '康全電訊', '所羅門', '福裕', '慧友', '高鋒', '京城', '弘憶股', '福大', '彬台', '翔耀', '花王', '新門', '訊舟', '志聖', '天品', '福懋油', '世紀', '全譜', '京晨科', '慶騰', '東捷', '均豪', '喬福', '永信建', '擎亞', '安國', '鑫科', '弘塑', '系微', '天方能源', '泰偉', '旺矽', '三地開發', '德晉', '聯鈞', '昇陽半導體', '鏵友益', '聯上發', '太普高', '天揚', '順藥', '昇益', '鑫龍騰', '藥華藥', '華友聯', '錦明', '泰金-KY', '鈊象', '和椿', '及成', '迎廣', '錸德', '宏碩系統', '訊聯基因', '光明', '藝舍-KY', '波力-KY', '華義', '華城', '益登', '富宇', '宏旭-KY', '雲豹能源', '天剛', '森寶', '大飲', '精湛', '勝昱', '盟立', '鈺邦', '坤悅', '精材', '佳能', '昇達科', '一詮', '達能', '晶悅', '峰源-KY', '上詮', '惠特', '加捷生醫', '愛山林', '友威科', '合騏', '理銘', '三發地產', '泰金寶-DR', '天良', '大城地產']


Industry Rankings:


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
95,Industrial Distribution,Industrials,158.72,129.60,93.18,94.38,"8374.TW,911608.TW,3114.TWO,2373.TW",99.0,98,46,16,1
1,Real Estate - Development,Real Estate,137.60,113.95,114.17,103.49,"2524.TW,5508.TWO,2537.TW,5455.TWO,3188.TWO,143...",98.0,94,99,93,2
29,Real Estate - Diversified,Real Estate,126.10,110.95,111.46,108.34,"1438.TW,9946.TW,6219.TWO,2520.TW,2545.TW,5512....",97.0,92,97,98,3
93,Software—Application,Technology,120.76,118.61,93.19,108.39,"6231.TWO,5203.TW,6902.TW",96.0,96,48,99,4
28,Utilities - Renewable,Utilities,119.93,124.15,103.27,104.05,"6869.TW,6873.TW,6806.TW,8087.TWO",95.0,97,89,94,5



number of filtered sectors: 5


***
Your "TWSE+TPEX_stocks_1y_20240808.csv" is in the "out" folder.
Your "TWSE+TPEX_industries_1y_20240808.csv" is in the "out" folder.
***

